## Exercise 8.1



In [15]:
import numpy as np
from scipy import signal
import pyfftw
import scipy.fftpack
import matplotlib.pyplot as plt
import pulse_shaping
import preamble_generator
import symbol_mod

# Generate ideal baseband signal
np.random.seed(2021) # seed to save the exact random numbers
N = 1000 # number of data bits
Bits = np.random.randint(0,2,N)  #random data
preamble = preamble_generator.preamble_generator()  
packet_bits = np.append(preamble, Bits)
preamble_length = len(preamble)
baseband_symbols = symbol_mod.symbol_mod(packet_bits, 'QPSK', preamble_length)
pulse_shape = 'rrc'
M = 8 #samples per symbol
fs = 1000000 #sampling rate in Hz
Ts = 1/fs
baseband = pulse_shaping.pulse_shaping(baseband_symbols, M, fs, pulse_shape, 0.9, 8)

frequency_offset = np.random.uniform(-0.01*fs,0.01*fs)
t =  np.arange(0,len(baseband)*Ts,Ts)
nonideal_term = np.exp(1j*2*np.pi*frequency_offset*t)
baseband_with_frequency_offset = np.multiply(baseband,nonideal_term)

segments_of_data_for_fft = baseband_with_frequency_offset # input data with frequency offset
num_fft_point = 800 # FFT length (increase till desired freq offset resolution obtained)
fs_in = fs*1.0 # ADC input sampling rate
    
spectrum = abs(pyfftw.interfaces.scipy_fftpack.fft(segments_of_data_for_fft, num_fft_point))

spectrum = np.fft.fftshift(spectrum)

peak_position = np.argmax(spectrum)

coarse_frequency = (peak_position-len(spectrum)/2) / len(spectrum) * fs_in

print("estimated frequency offset:", coarse_frequency)
print("actual frequency offset:", frequency_offset)

estimated frequency offset: 6250.0
actual frequency offset: 6506.152627238385


The screenshots below shot the estimation for different lengths of the FFT.

Length 10:

![FFT 10](FFT_10_81.PNG)

Length 100:

![FFT 100](FFT_100_81.PNG)

Length 200:

![FFT 200](FFT_200_81.PNG)

Length 500:

![FFT 500](FFT_500_81.PNG)

Length 800:

![FFT 800](FFT_800_81.PNG)


#### Observations

We can see that as the FFT length gets higher, the difference between the estimated offset and the actual offset gets closer. 

## Exercise 8.2


In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

import pulse_shaping
import preamble_generator
import symbol_mod

np.random.seed(2021)
N = 1000 # number of data bits
Bits = np.random.randint(0,2,N)
preamble = preamble_generator.preamble_generator()  
packet_bits = np.append(preamble, Bits)
preamble_length = len(preamble)
baseband_symbols = symbol_mod.symbol_mod(packet_bits, 'QPSK', preamble_length)
pulse_shape = 'rrc'
samples_perbit = 8 #samples per symbol
fs = 1000000 #sampling rate in Hz
Ts = 1/fs
baseband = pulse_shaping.pulse_shaping(baseband_symbols, samples_perbit, fs, pulse_shape, 0.9, 8)

frequency_offset = np.random.uniform(-0.01*fs,0.01*fs)
phase_offset = np.random.uniform(-np.pi,np.pi)
t = np.arange(0,len(baseband)*Ts,Ts)

nonideal_term = np.exp(1j*(2*np.pi*frequency_offset*t + phase_offset))
packet_data = np.multiply(baseband,nonideal_term) #packet data samples with frequency and phase 
Digital_LO = np.exp(1j*(-2*np.pi*frequency_offset*t)) # locally generated complex LO for frequency 
preamble_length = 180
payload_start = int(preamble_length*samples_perbit)

packet_data_freq_corrected = np.multiply(packet_data, Digital_LO)

packet_data_freq_corrected = packet_data_freq_corrected - np.mean(packet_data_freq_corrected\
                                                                  [payload_start:])
preamble = packet_data_freq_corrected[0:int(preamble_length*samples_perbit)]

angles = np.angle(preamble)

phase_estimated = np.mean(angles)

phase_corrected_packet = np.multiply(packet_data_freq_corrected, np.exp(-1j*phase_estimated))

plt.plot(np.real(packet_data[0:1700]))
plt.title('BB I channel before frequency and phase sync')
plt.show()

plt.plot(np.real(phase_corrected_packet[0:1700]))
plt.title('BB I channel after frequency and phase sync')
plt.show()

The image below is the output from running the code above.

![8.2](82.png)


#### Still need to answer the question:

- Screenshot of the plot before and after phase offset estimation. Why does the plot of I channel baseband symbols with phase and frequency offset have a region that looks like a sine wave? (Hint: Preamble is modulated OOK symbols + bunch of ones appended after.)

## Exercise 8.3


In [ ]:
import numpy as np
from scipy import signal
import pyfftw
import scipy.fftpack

import freq_sync

freqsync_data = np.load('FreqSync_data.npz') #load

segments_of_data_for_fft = freqsync_data['segments_of_data_for_fft']
num_fft_point = freqsync_data['coarse_fft_point']
fs_in = freqsync_data['fs_in']
coarse_frequency_desired = freqsync_data['coarse_frequency']

coarse_frequency = freq_sync.freq_sync(segments_of_data_for_fft, num_fft_point, fs_in)

print(np.array_equal(coarse_frequency, coarse_frequency_desired, equal_nan=False))

## Exercise 8.4



In [ ]:
import numpy as np
from scipy import signal

import phase_sync
    
phasesync_data = np.load('PhaseSync_data.npz') #load

packet_data = phasesync_data['payload_and_ones']
Digital_LO = phasesync_data['Digital_LO']
payload_start = phasesync_data['payload_start']
preamble_length = phasesync_data['ones_length']
samples_perbit = phasesync_data['samples_perbit']
phase_corrected_packet_desired = phasesync_data['payload_and_ones_corrected']

phase_corrected_packet = phase_sync.phase_sync(packet_data, Digital_LO, payload_start,preamble_length, samples_perbit)

# compare the obtained phase offset corrected packet with the desired packet
print(np.array_equal(phase_corrected_packet, phase_corrected_packet_desired, equal_nan=False))

## Exercise 8.5


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import time

def symbol_demod(baseband_symbols, scheme, channel_gain): #"a" is the bit array to be modulate

        a_demodulated = []
        
        if(scheme == 'OOK'):

                s_on = 1.0 * channel_gain

                s_off = 0* channel_gain

                baseband_symbols_I = baseband_symbols[0]

                baseband_symbols_Q = baseband_symbols[1]


                baseband_symbols_complex = baseband_symbols_I + 1j * baseband_symbols_Q


                for i in range( len(baseband_symbols_complex) ):

                                if (np.abs(baseband_symbols_complex[i] - s_on) < np.abs(baseband_symbols_complex[i] - s_off)):
                                       a_demodulated.append(1)
                                else:
                                       a_demodulated.append(0)


        if(scheme == 'BPSK'):

                baseband_symbols_I = baseband_symbols[0]

                baseband_symbols_Q = baseband_symbols[1]

                reference_plus = 1.0*channel_gain

                reference_minus = -reference_plus

                baseband_symbols_complex = baseband_symbols_I + 1j * baseband_symbols_Q

                for i in range(len(baseband_symbols_complex)):


                        if (np.abs(baseband_symbols_complex[i] - reference_plus) < np.abs(baseband_symbols_complex[i] - \
                                                                                          reference_minus)):
                               a_demodulated.append(0)
                        else:
                               a_demodulated.append(1)


        if(scheme == 'QPSK'):

                baseband_symbols_I = baseband_symbols[0]

                baseband_symbols_Q = baseband_symbols[1]


                I_demodulated = []
                Q_demodulated = []

                baseband_symbols_complex = baseband_symbols_I + 1j * baseband_symbols_Q

                reference_00 = -1.0*channel_gain  -1j* channel_gain

                reference_11 = 1.0*channel_gain + 1j* channel_gain

                reference_01 = -1.0*channel_gain + 1j* channel_gain

                reference_10 = 1.0*channel_gain  -1j* channel_gain

                for i in range(len(baseband_symbols_complex)):

                        symbol = baseband_symbols_complex[i]

                        if (  np.abs(symbol - reference_11) == np.amin(  [np.abs(symbol - reference_11),  \
                                                                          np.abs(symbol - reference_10), \
                               I_demodulated.append(1)
                               Q_demodulated.append(1)
                        elif(np.abs(symbol - reference_10) == np.amin(  [np.abs(symbol - reference_11),  \
                                                                          np.abs(symbol - reference_10), \
                               I_demodulated.append(1)
                               Q_demodulated.append(0)
                        elif(np.abs(symbol - reference_01) == np.amin(  [np.abs(symbol - reference_11),  \
                                                                          np.abs(symbol - reference_01)]  )  ):
                               I_demodulated.append(0)
                               Q_demodulated.append(1)
                        elif(np.abs(symbol - reference_00) == np.amin(  [np.abs(symbol - reference_11),  \
                                                                          np.abs(symbol - reference_01)]  )  ):
                               I_demodulated.append(0)
                               Q_demodulated.append(0)

                a_demodulated = np.append(I_demodulated, Q_demodulated)


        return a_demodulated

def rotate(vector, angle):

        x = np.real(vector)

        y = np.imag(vector)

        x_new = np.cos(angle)*x - np.sin(angle)*y

        y_new = np.sin(angle)*x + np.cos(angle)*y

        vector_new = x_new + 1j*y_new
        

        return vector_new

qpskdemod_data = np.load('SymbolDemodResult.npz') #load

baseband_symbols = qpskdemod_data['baseband_symbols']
scheme = qpskdemod_data['scheme']
channel_gain = qpskdemod_data['channel_gain']
demod_bits = qpskdemod_data['a_demodulated']

demodulated_bits = symbol_demod(baseband_symbols, scheme, channel_gain)

print(np.array_equal(demodulated_bits, demod_bits, equal_nan=False))

## Exercise 8.6

The image below is the BER for wired connection:

![Wired](Wired.PNG)

The image below is the BER for wireless connection:

![Wireless](Wireless.PNG)


It is important to notice that even though the reported BER is not zero, the actual data sent and received matches. The non-zero BER reported is probably caused by a bug in the code.

### Reflection Questions - Alex

#### Q1: How is what you are doing relevant to what you are learning?
This week in lab we explored some more advanced methods for transmitting and demodulating bits. This is very relevant as we're implementing what we directly talked about and explored in lab like QPSK demodulation.

#### Q2: How can you relate this to the real world? Give specific examples where this information can be useful.
The QPSK symbol modulation allows for more data to be sent using the same amount of bandwidth compared to simple PAM modulating. In the real world bandwidth is very precious, and the more data that can get sent through the better. Thus what we did in lab is essential to the real world.

#### Q3: What was not so successful for you? Why?
This week we had some issues with some of the code being incorrect the first time we worked through it. On the other hand, the hardware was the same as it has been and didn't have any issues. However, it did take a bit of time to diagnose where exactly the issue was. My two takeaways from this are 1) that it's important to have individual tests on different sections of the complete system to help isolate where issues are. My second observation is that with the same hardware setup we have, it is possible to transmit and demodulate a lot of different types of signals with varying levels of efficiency and error probabilities.

#### Q4: What would you like to improve upon? How can you make these changes?
While there are other things I would like to tackle for the final projects, I would find enjoyment in testing out different constellations to see how effective they are against different merit metrics (error probability/throughput). I would find it interesting to test different ones out in practice to see how closely different setups match up to what is expected mathematically.

### Reflection Questions - David

#### Q1: What are you being asked to do and why?
In this lab we started covering two algorithms for both frequency and phase offset estimation. The frequency offset estimation relied on using the FFT of a known preamble and look fo the matching peak in the data we are receiving. The phase offset estimation relied on calculating the average of the phase angles of the preamble samples. We then integrated both algorithms and tested in a transmitter-receiver system for both wired and wireless communication.

#### Q2: What do you think about what you see and do?
We saw how the algorithms introduce trade-offs in our design. For example, in the frequency offset estimation, one way of increasing the accuracy of the estimation is to increase the length of the FFT (which introduces more computational complexity) and/or to increase the length of the preamble (which reduces the effective transmission rate of the system). This trade-offs need to be weighted depending on the application. For example, maybe on a wireless communication channel we need to take a longer FFT than on wired. 

#### Q3: What was not so successful for you? Why?
We were having some problems with the code for the last part of the lab. We thought our system was poorly connected since the reported BER was different than zero. But upon closer examination we noticed that the messages sent and received were the same, and that the issue was a bug in the code that reported the wrong BER value.

#### Q4: How is what you are doing relevant to what you are learning?
Frequency and phase offsets introduce significant distortions in our signals as we saw in previous labs and lectures. If we want to build a robust communication system were the receiver and transmitter do not need to have a shared oscillator, we are highly interested in these mathematical tools to properly estimate the frequency and phase offset and compensate for them.

